In [19]:
%%capture --no-display
%pip install requests
%pip install python-dotenv
%pip install numpy

In [3]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def read_input_url(url):
    headers = {
        'Cookie': f'session={os.getenv("SESSION_COOKIE")}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text

In [4]:
"""Day 1 input"""
day_1_input_url = "https://adventofcode.com/2024/day/1/input"
day_3_content = read_input_url(day_1_input_url)
 
left = []
right = []
for line in day_3_content.splitlines():
    left_num, right_num = line.split("   ")
    left.append(int(left_num))
    right.append(int(right_num))

In [5]:
"""Solution to Day 1 Puzzle 1"""
def solution_1_1(left, right):
    similarity_score = 0
    for i in range(len(left)):
        similarity_score += abs(left[i] - right[i])
    return similarity_score

solution_1_1(left, right)

29037157

In [6]:
"""Solution to Day 1 Puzzle 2"""
def solution_1_2(left, right):
    similarity_score = 0
    for i in range(len(left)):
        left_num = left[i]
        occurences = right.count(left_num)
        similarity_score += left_num * occurences if occurences > 0 else 0
    return similarity_score

solution_1_2(left, right)

19678534

In [7]:
"""Day 2 input"""
day_2_input_url = "https://adventofcode.com/2024/day/2/input"
day_3_content = read_input_url(day_2_input_url)

# A report is a list of levels seperated by new line
# A level is a number, seperasted by space

list_of_reports = []
for line in day_3_content.splitlines():
    report = line.split(" ")
    report = [int(level) for level in report]
    list_of_reports.append(report)

In [8]:
"""Soltion to Day 2 Puzzle 1"""

def is_ascending(report):
    return all(report[i] <= report[i+1] for i in range(len(report)-1))
    
def is_descending(report):
    return all(report[i] >= report[i+1] for i in range(len(report)-1))
    
def diff_within(report, min, max):
    for i in range(len(report)-1):
        diff = abs(report[i] - report[i+1])
        if diff < min or diff > max:
            return False
            
    return True

def is_safe(report):
    if not (is_ascending(report) or is_descending(report)):
        return False
    return diff_within(report, 1, 3)

count_safe_reports = 0
for report in list_of_reports:
    if is_safe(report):
        count_safe_reports += 1

count_safe_reports

359

In [9]:
"""Soltion to Day 2 Puzzle 2"""

def is_safe_with_tolerance_of_one_level_removed(report):
    for i in range(len(report)):
        new_report = report[:i] + report[i+1:]
        if is_safe(new_report):
            return True
    return False

count_safe_reports_with_tolerance_of_one_removed = 0
for report in list_of_reports:
    if is_safe_with_tolerance_of_one_level_removed(report):
        count_safe_reports_with_tolerance_of_one_removed += 1

count_safe_reports_with_tolerance_of_one_removed

418

In [10]:
"""Day 3 input"""
day_3_input_url = "https://adventofcode.com/2024/day/3/input"
day_3_content = read_input_url(day_3_input_url)

In [11]:
"""Solution to Day 3 Puzzle 1"""

import re

def parse(content, pattern):
    return re.findall(pattern, content)

def evaluate(parsed_content):
    result = 0
    for match in parsed_content:
        result += int(match[0]) * int(match[1])
    return result

def extract_mul_instructions(content, pattern):
    parsed_content = parse(content, pattern)
    return evaluate(parsed_content)


mul_pattern = r"mul\((\d+),(\d+)\)"
extract_mul_instructions(day_3_content, mul_pattern)

164730528

In [12]:
"""Solution to Day 3 Puzzle 2"""

def build_pattern():
    patterns = {
        'do': r'do\(\)',
        'dont': r'don\'t\(\)',
        'mul': r"mul\((\d+),(\d+)\)"
    }
    
    combined = '|'.join(f'(?P<{name}>{pattern})' 
                       for name, pattern in patterns.items())
    
    return re.compile(combined)


def extract_do_and_dont_mul_instructions(content):
    combined_pattern = build_pattern()
    pattern = re.compile(combined_pattern)
    
    enabled = True
    result = 0
    
    for match in pattern.finditer(content):
        if match.group('do'):
            enabled = True
        elif match.group('dont'):
            enabled = False
        elif match.group('mul') and enabled:
            a, b = map(int, match.group('mul')[4:-1].split(','))
            result += a * b
            
    return result
        

extract_do_and_dont_mul_instructions(day_3_content)

70478672

In [13]:
"""Day 4 input"""
day_4_input_url = "https://adventofcode.com/2024/day/4/input"
day_4_content = read_input_url(day_4_input_url)

In [ ]:
"""Solution to Day 4 Puzzle 1"""

import re
import numpy as np
from enum import Enum, auto


class Direction(Enum):
    HORIZONTAL = auto()
    VERTICAL = auto()
    LEFT_DIAGONAL = auto()
    RIGHT_DIAGONAL = auto()
    REVERSE = auto()


def create_matrix(input_str):
    return np.array([list(line) for line in input_str.splitlines()])

def get_all_lines(matrix, word_length, directions):
    rows, cols = matrix.shape
    lines = []

    if Direction.HORIZONTAL in directions:
        for row in matrix:
            lines.append("".join(row))

    if Direction.VERTICAL in directions:
        for col in matrix.T:
            lines.append("".join(col))

    if Direction.LEFT_DIAGONAL in directions:
        for i in range(-(rows - word_length), cols - word_length + 1):
            diag = np.diag(matrix, k=i)
            if len(diag) >= word_length:
                lines.append("".join(diag))

    if Direction.RIGHT_DIAGONAL in directions:
        flipped = np.fliplr(matrix)
        for i in range(-(rows - word_length), cols - word_length + 1):
            diag = np.diag(flipped, k=i)
            if len(diag) >= word_length:
                lines.append("".join(diag))

    if Direction.REVERSE in directions:
        lines.extend([line[::-1] for line in lines[:]])

    return lines


def count_word_occurrences(input_str, word, directions=None):
    matrix = create_matrix(input_str)
    lines = get_all_lines(matrix, len(word), directions)
    total = 0
    pattern = f"(?={word})"

    for line in lines:
        total += len(re.findall(pattern, line))

    return total

occurences = count_word_occurrences(
    day_4_content,
    "XMAS",
    {Direction.HORIZONTAL, Direction.REVERSE, Direction.LEFT_DIAGONAL, Direction.RIGHT_DIAGONAL, Direction.VERTICAL},
)

occurences

2718

In [ ]:
"""Solution to Day 4 Puzzle 2"""

def get_crossed_diagonal_occurrences(content, word):
    matrix = np.array([list(line) for line in content.splitlines()])
    rows, cols = matrix.shape
    occurrences = 0

    word_len = len(word)
    center_idx = word_len // 2
    center_char = word[center_idx]
    pattern = f"(?={word})"

    # Find all center character positions
    center_positions = np.argwhere(matrix == center_char)

    for pos in center_positions:
        i, j = pos

        # Skip if too close to edges
        if (
            i < center_idx
            or i >= rows - center_idx
            or j < center_idx
            or j >= cols - center_idx
        ):
            continue

        # Extract longer diagonals for pattern matching
        left_diag = "".join(
            matrix[
                i - center_idx : i + center_idx + 1, j - center_idx : j + center_idx + 1
            ].diagonal()
        )
        right_diag = "".join(
            np.fliplr(
                matrix[
                    i - center_idx : i + center_idx + 1,
                    j - center_idx : j + center_idx + 1,
                ]
            ).diagonal()
        )

        # Check both directions in each diagonal
        left_matches = len(re.findall(pattern, left_diag)) + len(
            re.findall(pattern, left_diag[::-1])
        )
        right_matches = len(re.findall(pattern, right_diag)) + len(
            re.findall(pattern, right_diag[::-1])
        )

        if left_matches > 0 and right_matches > 0:
            occurrences += 1

    return occurrences


get_crossed_diagonal_occurrences(day_4_content, "MAS")

2046